In [1]:
import numpy as np
indices = np.asarray([2, 0, 3, 1], dtype="uint32")
values = np.asarray([0.0, 1.0, 2.0, 3.0], dtype="float32")
from unray import Data, DataDisplay

In [2]:
idata = Data(array=indices)

In [3]:
iw = DataDisplay(data=idata)

In [4]:
iw

A Jupyter Widget

In [5]:
vdata = Data(array=values)

In [6]:
vw = DataDisplay(data=vdata)

In [7]:
vw

A Jupyter Widget

In [8]:
indices[1], indices[0] = indices[0], indices[1]
idata.send_state()

In [9]:
values[2], values[3] = values[3], values[2]
vdata.send_state()